# LangChain Agents!

NOTE: You will need a Google Search API Key.
You can get a key for free from https://serpapi.com/

Make sure that you have an environment variable named: SERPAPI_API_KEY
with the value being the value of your key

In [1]:
pip -q install langchain openai google-search-results tiktoken wikipedia numexpr


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

In [3]:
llm = OpenAI(temperature=0)

/var/folders/fn/f1_5kmtj0wd5w15pp0qnlf780000gn/T/ipykernel_87243/3183534344.py:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [4]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [5]:
# What is a tool like this
tools[0].name, tools[0].description

('Search',
 'A search engine. Useful for when you need to answer questions about current events. Input should be a search query.')

In [6]:
agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True,
                         handle_parsing_errors=True)

/var/folders/fn/f1_5kmtj0wd5w15pp0qnlf780000gn/T/ipykernel_87243/1334005195.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools,


In [7]:
# take a look at the template!
# NOTE!!!! : The agent already has built up a prompt template for us based on the description of the tool
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the ori

# Chain Of Thought

Running these question on the agents will cause it to get through a series of observations and thoughts.
See below!

In [8]:
agent.run("Hi, How are you today?")

/var/folders/fn/f1_5kmtj0wd5w15pp0qnlf780000gn/T/ipykernel_87243/4271694833.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Hi, How are you today?")




> Entering new AgentExecutor chain...
 This is not a question that requires a specific tool to answer.
Final Answer: I am an AI and do not have emotions, so I am always ready to assist you with any questions you may have.

> Finished chain.


'I am an AI and do not have emotions, so I am always ready to assist you with any questions you may have.'

In [9]:

agent.run("Who is the current president of the United States?")




> Entering new AgentExecutor chain...
 I should use the search engine to find the answer.
Action: Search
Action Input: "current president of the United States"
Observation: Donald Trump
Thought: I should double check to make sure this is the most recent information.
Action: Search
Action Input: "who is the current president of the United States"
Observation: Donald Trump
Thought: I should also check for any recent news articles about the president.
Action: Search
Action Input: "recent news articles about the president of the United States"
Observation: [{'title': "Donald Trump orders inquiry into Joe Biden's actions as president, alleging 'cognitive decline'", 'link': 'https://www.bbc.com/news/articles/cvg9z9veyr1o', 'source': 'BBC', 'source_logo': 'https://serpapi.com/searches/6841f869e171b937ed4308c6/images/956968f488893ea63986f094b18cd8ddc7ce915951214f3e0c2c85b1889ffcf3.png', 'date': '15 hours ago', 'thumbnail': 'https://serpapi.com/searches/6841f869e171b937ed4308c6/images/956968f

'The current president of the United States is Donald Trump.'

In [10]:
agent.run("What is the average age in the United States?")




> Entering new AgentExecutor chain...
 I should use a search engine to find the answer.
Action: Search
Action Input: "average age United States"
Observation: ["The nation's median age increased by 0.2 years to 38.9 years between 2021 and 2022, according to Vintage 2022 Population Estimates released ...", 'In 2023, the median age of the population of the United States was 39.2 years. While this may seem quite young, the median age in 1960 was even ...', 'Median Age. The median age in the United States is 38.5 years (2025).', 'Age and sex distribution ; 0–14 years, 18.2% ; 15–24 years, 13.0% ; 25–54 years, 39.0% ; 55–64 years, 12.9%.', "New census data shows that the country's median age is now over 38. In 1980, it was 30.", 'In 2005, it was 28.1 years. Among the 216 countries evaluated here, the USA are in 64th place with an median age of 38.9 years, led by Monaco at 56.9 years.', 'In 2022 the U.S. had a median age of 39, but the median age by state ranged from the low 30s to the mid 

'The average age in the United States is 38.85 years.'

# Smarter Agent

Let's make an even smarter agent that can do more things!

In [ ]:
tools = load_tools(["serpapi", "llm-math","wikipedia","terminal"], llm=llm, allow_dangerous_tools=True)
# NOTE: terminal tool is dangerous, use with caution!!!!

In [14]:
agent = initialize_agent(tools, 
                         llm, 
                         agent="zero-shot-react-description", 
                         verbose=True)

In [15]:
agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nSearch(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.\nwikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nterminal - Run shell commands on this MacOS machine.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be

In [16]:

agent.run("Where is Apple's office headquarters?")



> Entering new AgentExecutor chain...
 I should use the wikipedia tool to search for information about Apple's headquarters.
Action: wikipedia
Action Input: "Apple headquarters"
Observation: Page: Apple Park
Summary: Apple Park, also known as Apple Campus 2, is the corporate headquarters of Apple Inc., located in Cupertino, California, United States. It was opened to employees in April 2017, while construction was still underway. It replaced Apple Campus as the company's corporate headquarters.
The main building's scale and circular groundscraper design, by Norman Foster, has earned the structure the media nickname "the spaceship". Located on a suburban site totaling 1.46 km2 (360 acres), it houses more than 12,000 employees in one central four-story circular building of approximately 0.26 km2 (64 acres). Apple co-founder Steve Jobs wanted the campus to look less like a business park and more like a nature refuge; 80 percent of the site consists of green space planted with drought-re

"Apple's office headquarters is located at Apple Park in Cupertino, California."

In [17]:

agent.run("What files are in my current directory?")



> Entering new AgentExecutor chain...
 I should use the terminal to run a shell command
Action: terminal
Action Input: lsExecuting command:
 ls

Observation: 01-LangChain_Basics.ipynb
04-LangChain_Bind_Functions.ipynb
05-LangChainAgents.ipynb
06-LangChain_PDF_Extraction.ipynb
Chinook.db
Chinook_Sqlite.sql
LangChain.pdf
README.md
chinook-db.png
constitution.pdf
embedding1.png
embedding2.png
embedding3.png
langchain_chroma_test.py
langchain_youtube.py

Thought:

/Users/justingrammens/dev/BSC/.venv/lib/python3.13/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


 I now know the final answer
Final Answer: There are 14 files in the current directory.

> Finished chain.


'There are 14 files in the current directory.'

In [19]:
agent.run("Does my current directory have python notebook files?")



> Entering new AgentExecutor chain...
 I should use the terminal to check the current directory
Action: terminal
Action Input: lsExecuting command:
 ls

Observation: 01-LangChain_Basics.ipynb
04-LangChain_Bind_Functions.ipynb
05-LangChainAgents.ipynb
06-LangChain_PDF_Extraction.ipynb
Chinook.db
Chinook_Sqlite.sql
LangChain.pdf
README.md
chinook-db.png
constitution.pdf
embedding1.png
embedding2.png
embedding3.png
langchain_chroma_test.py
langchain_youtube.py

Thought:

/Users/justingrammens/dev/BSC/.venv/lib/python3.13/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


 I should check if there are any python notebook files in the current directory
Action: Search
Action Input: "python notebook files in current directory"
Observation: ['You can use os.getcwd (current working directory) or in the native os command pwd . In [8]: import os In [9]: os.getcwd() Out[9]: ...', 'I want to become fluent with entering code in JN to successfully navigate my directories. For example, I want to import a .csv file like this:', "Open anaconda prompt and type cd 'D:/directory/of/your/choice' afterwards type jupyter notebook or jupyter lab depending on your preference.", 'The os.getcwd() method returns the current working directory (CWD) of the current process. In this case, the current process is the Jupyter ...', 'You can upload files to a Jupyter notebook directory (from the classic notebook homepage, for example), or pull in files using tools like nbgitpuller.', "In this post I'll discuss how to change the Jupyter notebook startup folder in Anaconda which is instal

/Users/justingrammens/dev/BSC/.venv/lib/python3.13/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


 I now know the final answer
Final Answer: Yes, there are python notebook files in the current directory.

> Finished chain.


'Yes, there are python notebook files in the current directory.'